# Task 1

### Data processing

In [1]:
pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Download raw data
data = pd.read_csv('data/Groceries data train.csv')

In [4]:
data

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3021,30/01/2015,frankfurter,2015,1,30,4
1,1292,24/10/2015,pork,2015,10,24,5
2,4206,4/04/2014,root vegetables,2014,4,4,4
3,4369,25/08/2015,onions,2015,8,25,1
4,1522,1/07/2014,waffles,2014,7,1,1
...,...,...,...,...,...,...,...
26995,4206,14/01/2014,rolls/buns,2014,1,14,1
26996,1660,15/06/2014,domestic eggs,2014,6,15,6
26997,1595,10/05/2015,liver loaf,2015,5,10,6
26998,3231,31/03/2015,sausage,2015,3,31,1


In [5]:
data.columns

Index(['Member_number', 'Date', 'itemDescription', 'year', 'month', 'day',
       'day_of_week'],
      dtype='object')

In [8]:
# Group the data by 'Member_number' and 'Date', then aggregate the 'itemDescription' into lists
transactions_data = data.groupby(['Member_number'])['itemDescription'].apply(list).reset_index()

# Extract the list of transactions
transactions = transactions_data['itemDescription'].tolist()

sorted_transactions = [sorted(list(set(transaction))) for transaction in transactions]

transactions = sorted_transactions

transactions = [transaction for transaction in transactions if len(transaction) > 1]

# Display the first transaction as an example
transactions[0]

['canned beer',
 'misc. beverages',
 'pastry',
 'pickled vegetables',
 'sausage',
 'soda',
 'yogurt']

In [9]:
print(type(transactions))

<class 'list'>


### Mine frequent patterns by using Apriori algorithm

In [10]:
pip install apyori

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
from apyori import apriori
from mlxtend.frequent_patterns import association_rules
rules = apriori(transactions, min_support=0.002, min_confidence=0.8,min_lift=1, min_length=1)
Results = list(rules)
Results

[RelationRecord(items=frozenset({'bottled water', 'whole milk', 'canned vegetables'}), support=0.002425222312045271, ordered_statistics=[OrderedStatistic(items_base=frozenset({'bottled water', 'canned vegetables'}), items_add=frozenset({'whole milk'}), confidence=0.8181818181818182, lift=2.2760665121984465)]),
 RelationRecord(items=frozenset({'spices', 'bottled water', 'whole milk'}), support=0.002425222312045271, ordered_statistics=[OrderedStatistic(items_base=frozenset({'spices', 'bottled water'}), items_add=frozenset({'whole milk'}), confidence=0.8181818181818182, lift=2.2760665121984465)]),
 RelationRecord(items=frozenset({'dishes', 'butter', 'whole milk'}), support=0.002155753166262463, ordered_statistics=[OrderedStatistic(items_base=frozenset({'dishes', 'butter'}), items_add=frozenset({'whole milk'}), confidence=1.0, lift=2.7818590704647677)]),
 RelationRecord(items=frozenset({'canned fish', 'newspapers', 'whole milk'}), support=0.002425222312045271, ordered_statistics=[OrderedSt

In [27]:
df_results = pd.DataFrame(Results)
df_results

,items,support,ordered_statistics
0,"(bottled water, whole milk, canned vegetables)",0.002425,"[((bottled water, canned vegetables), (whole m..."
1,"(spices, bottled water, whole milk)",0.002425,"[((spices, bottled water), (whole milk), 0.818..."
2,"(dishes, butter, whole milk)",0.002156,"[((dishes, butter), (whole milk), 1.0, 2.78185..."
3,"(canned fish, newspapers, whole milk)",0.002425,"[((canned fish, newspapers), (whole milk), 0.8..."
4,"(white bread, napkins, whole milk)",0.004581,"[((white bread, napkins), (whole milk), 0.8095..."
...,...,...,...
81,"(bottled water, whole milk, pip fruit, other v...",0.002695,"[((other vegetables, yogurt, pip fruit, bottle..."
82,"(bottled water, soda, rolls/buns, whipped/sour...",0.002425,"[((bottled water, rolls/buns, whipped/sour cre..."
83,"(shopping bags, soda, rolls/buns, whole milk, ...",0.002695,"[((shopping bags, rolls/buns, soda, canned bee..."
84,"(frozen vegetables, rolls/buns, other vegetabl...",0.002425,"[((other vegetables, frozen vegetables, yogurt..."


In [28]:
# keep support in a separate data frame so we can use later
support = df_results.support

In [29]:
#all four empty list which will contain lhs, rhs, confidance and lift respectively.
first_values = []
second_values = []
third_values = []
fourth_value = []

# loop number of rows time and append 1 by 1 value in a separate list
# first and second element was frozenset which need to be converted in list
for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])

In [36]:
# convert all four list into dataframe for further operation
lhs = pd.DataFrame(first_values)
rhs = pd.DataFrame(second_values)

confidance=pd.DataFrame(third_values,columns=['Confidance'])

lift=pd.DataFrame(fourth_value,columns=['lift'])

In [59]:
# concat all list together in a single dataframe
df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final
df_final.fillna(value=' ', inplace=True)
df_final.head()

,0,1,2,3,0,support,Confidance,lift
0,bottled water,canned vegetables,,,whole milk,0.002425,0.818182,2.276067
1,spices,bottled water,,,whole milk,0.002425,0.818182,2.276067
2,dishes,butter,,,whole milk,0.002156,1.000000,2.781859
3,canned fish,newspapers,,,whole milk,0.002425,0.818182,2.276067
4,white bread,napkins,,,whole milk,0.004581,0.809524,2.251981


In [60]:
#set column name
df_final.columns = ['lhs',1,2,3,'rhs','support','confidance','lift']
df_final.head()

,lhs,1,2,3,rhs,support,confidance,lift
0,bottled water,canned vegetables,,,whole milk,0.002425,0.818182,2.276067
1,spices,bottled water,,,whole milk,0.002425,0.818182,2.276067
2,dishes,butter,,,whole milk,0.002156,1.000000,2.781859
3,canned fish,newspapers,,,whole milk,0.002425,0.818182,2.276067
4,white bread,napkins,,,whole milk,0.004581,0.809524,2.251981


In [61]:
# add all three column to lhs itemset only
df_final['lhs'] = df_final['lhs'] + str(", ")  + df_final[1] + df_final[2] 

df_final['rhs'] = df_final['rhs'] + str(", ") + df_final[3]

In [62]:
df_final.drop(columns=[1,2,3],inplace=True)

In [63]:
df_final.sort_values('lift', ascending=False).head(10)

,lhs,rhs,support,confidance,lift
27,"waffles, bottled waterwhipped/sour cream","tropical fruit,",0.002156,0.888889,5.294810
21,"specialty chocolate, bottled watercanned beer","root vegetables,",0.002156,0.888889,5.051557
32,"pork, whole milkcandy","yogurt,",0.002156,0.888889,4.175527
77,"berries, tropical fruitother vegetables","yogurt, whole milk",0.002425,0.818182,3.843383
6,"UHT-milk, sodafrankfurter","rolls/buns,",0.002156,1.000000,3.666996
59,"frankfurter, sausagepastry","soda,",0.002425,0.818182,3.362428
35,"domestic eggs, shopping bagscanned beer","soda,",0.002425,0.818182,3.362428
84,"other vegetables, frozen vegetablesyogurt","rolls/buns, sausage",0.002425,0.900000,3.300296
44,"frozen meals, citrus fruitwhole milk","rolls/buns,",0.002695,0.833333,3.055830
60,"frozen vegetables, frozen mealswhole milk","other vegetables,",0.002156,0.888889,3.037446


In [64]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [65]:
df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,True,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3707,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3708,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3709,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [84]:
frequent_itemsets = fpgrowth(df, min_support=0.002, use_colnames=True)
frequent_itemsets

from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(soda),(whole milk),0.243331,0.359472,0.094314,0.387597,1.078240,0.006844,1.045926,0.095897
1,(whole milk),(soda),0.359472,0.243331,0.094314,0.262369,1.078240,0.006844,1.025810,0.113286
2,(rolls/buns),(soda),0.272703,0.243331,0.072487,0.265810,1.092383,0.006130,1.030618,0.116280
3,(soda),(rolls/buns),0.243331,0.272703,0.072487,0.297896,1.092383,0.006130,1.035882,0.111766
4,(other vegetables),(soda),0.292643,0.243331,0.075182,0.256906,1.055790,0.003973,1.018269,0.074704
...,...,...,...,...,...,...,...,...,...,...
16693,(potato products),(rolls/buns),0.004581,0.272703,0.002156,0.470588,1.725645,0.000907,1.373784,0.422442
16694,(brandy),(bottled water),0.007006,0.165993,0.002156,0.307692,1.853646,0.000993,1.204677,0.463772
16695,(brandy),(rolls/buns),0.007006,0.272703,0.002156,0.307692,1.128306,0.000245,1.050540,0.114518
16696,(brandy),(whole milk),0.007006,0.359472,0.003234,0.461538,1.283935,0.000715,1.189552,0.222705


In [96]:
def predict_items(purchased_items):
    # create a list to store the predicted items
    predicted_items = []
    
    # iterate over the top rules
    for index, row in rules.iterrows():
        # get the items in the antecedent and consequent of the rule
        antecedent = row["antecedents"]
        consequent = row["consequents"]
        
        # check if all the items in the antecedent are in the purchased items
        if antecedent.issubset(purchased_items):
            # add the items in the consequent to the predicted items
            for item in consequent:
                if item not in purchased_items and item not in predicted_items:
                    predicted_items.append(item)
    
    # return the list of predicted items
    return predicted_items
    
    # return the list of predicted items
    return predicted_items

In [97]:
# make a prediction for a sample set of purchased items
purchased_items = {"yogurt", "soda"}
predicted_items = predict_items(purchased_items)

# print the predicted items to the console
print(predicted_items)

['whole milk', 'rolls/buns', 'other vegetables', 'sausage', 'bottled water']


In [98]:
# make a prediction for a sample set of purchased items
purchased_items = {"yogurt"}
predicted_items = predict_items(purchased_items)

# print the predicted items to the console
print(predicted_items)

['soda', 'whole milk', 'rolls/buns', 'other vegetables', 'sausage']
